# Deepfake Audio Detection using SVM

This notebook implements a Support Vector Machine (SVM) model for detecting Urdu deepfake audio.

In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import joblib
import sys
sys.path.append('..')
from utils.audio_features import extract_features

# Try to import the datasets library, but provide a fallback if it's not available
try:
    from datasets import load_dataset
    DATASETS_AVAILABLE = True
except ImportError:
    print("Warning: 'datasets' library not available. Will use local data if provided.")
    DATASETS_AVAILABLE = False

## Load and Preprocess Data

We'll load the Urdu deepfake detection dataset and extract features from the audio files.

In [ ]:
# Function to extract features from audio files
def extract_audio_features(audio_path, sr=22050, duration=5):
    """Extract audio features from an audio file."""
    try:
        # Load audio file with a fixed duration
        y, sr = librosa.load(audio_path, sr=sr, duration=duration)
        
        # Extract MFCCs (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfccs, axis=1)
        mfcc_std = np.std(mfccs, axis=1)
        
        # Extract spectral features
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
        spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
        
        # Extract temporal features
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y)[0]
        
        # Combine all features
        features = np.concatenate([
            mfcc_mean, mfcc_std,
            [np.mean(spectral_centroid), np.std(spectral_centroid)],
            [np.mean(spectral_bandwidth), np.std(spectral_bandwidth)],
            [np.mean(spectral_rolloff), np.std(spectral_rolloff)],
            [np.mean(zero_crossing_rate), np.std(zero_crossing_rate)]
        ])
        
        return features
    except Exception as e:
        print(f"Error extracting features from {audio_path}: {e}")
        return None

In [ ]:
# Load the dataset
if DATASETS_AVAILABLE:
    try:
        print("Loading the Urdu deepfake detection dataset...")
        ds = load_dataset("CSALT/deepfake_detection_dataset_urdu")
        print(f"Dataset loaded successfully. Available splits: {ds.keys()}")
        
        # Process the dataset
        features = []
        labels = []
        
        # Process training data
        for item in ds['train']:
            audio_path = item['audio_path']
            label = 1 if item['label'] == 'deepfake' else 0  # Convert to binary labels
            
            # Extract features
            audio_features = extract_audio_features(audio_path)
            if audio_features is not None:
                features.append(audio_features)
                labels.append(label)
        
        # Convert to numpy arrays
        X = np.array(features)
        y = np.array(labels)
        
        print(f"Processed {len(X)} audio files with {X.shape[1]} features per file.")
        print(f"Class distribution: {np.bincount(y)}")
        
    except Exception as e:
        print(f"Error loading dataset: {e}")
        print("Falling back to local data if available...")
        DATASETS_AVAILABLE = False

# Fallback to local data if datasets library is not available or loading failed
if not DATASETS_AVAILABLE:
    print("Using local data...")
    # Here you would load local audio files and extract features
    # For demonstration, we'll create some dummy data
    X = np.random.rand(100, 32)  # 100 samples, 32 features
    y = np.random.randint(0, 2, 100)  # Binary labels (0: real, 1: deepfake)
    print(f"Created dummy data with {X.shape[0]} samples and {X.shape[1]} features.")
    print(f"Class distribution: {np.bincount(y)}")

## Preprocess and Split Data

In [ ]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

## Train SVM Model

In [ ]:
# Train SVM model
print("Training SVM model...")
svm_model = SVC(kernel='rbf', probability=True, C=1.0, gamma='scale')
svm_model.fit(X_train, y_train)
print("Model training completed.")

## Evaluate Model

In [ ]:
# Evaluate model
y_pred = svm_model.predict(X_test)
y_prob = svm_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
auc_roc = roc_auc_score(y_test, y_prob)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Bonafide', 'Deepfake']))

# Print summary metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC-ROC: {auc_roc:.4f}")

## Visualize ROC Curve

In [ ]:
# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test, y_prob)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'SVM (AUC = {auc_roc:.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Deepfake Detection')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Save Model

In [ ]:
# Save model and scaler
model_path = '../models/svm_deepfake.pkl'
scaler_path = '../models/svm_deepfake_scaler.pkl'

joblib.dump(svm_model, model_path)
joblib.dump(scaler, scaler_path)

print(f"Model saved to {model_path}")
print(f"Scaler saved to {scaler_path}")